In [1]:
import pandas as pd
import numpy as np 
from google.cloud import bigquery
from utils import read_and_render
from UKBInfoMapper import UKBInfoMapper
mapper_info = UKBInfoMapper()

In [2]:
client = bigquery.Client()
path = 'sql/sql_file.sql'
query = read_and_render(path)
result = client.query(query).to_dataframe()
#120107 missing

In [3]:
result.outcome.value_counts()

0    27456
1     2101
Name: outcome, dtype: Int64

In [4]:
result.columns = result.columns + '_'
result = mapper_info._rename_columns_on_data(df=result,original_col="field_id",new_col="field_name")

In [5]:
def convert_to_datetime(df):
    list_columns_date = ['DateLostToFollow-Up','DateOfAttendingAssessmentCentre','DateOfDeath','outcome_date_','max_date_']
    for col in list_columns_date: 
        df[col] = pd.to_datetime(df[col])
    return df 
result = convert_to_datetime(result)
list_columns_date = ['DateLostToFollow-Up','DateOfDeath','outcome_date_','max_date_']
for col in list_columns_date:
    result[col] = (result[col] - result['DateOfAttendingAssessmentCentre'])/pd.to_timedelta('365 days')
result['survival_time'] = np.nanmin(result[list_columns_date],1)

In [8]:
list_non_feature = ['DateLostToFollow-Up','DateOfAttendingAssessmentCentre','DateOfDeath','outcome_date_']
list_features = result.columns

In [9]:
# Fillnan with mode/median
# Encoding
# Cox Model

In [10]:
result.groupby('_eid').max().outcome.value_counts()

KeyError: '_eid'